In [292]:
import numpy as np
from scipy.stats import norm
from scipy.stats import binom
from tqdm import tqdm

states = [
    {
        "state": "Pennsylvania",
        "dem": 48.2,
        "rep": 47.0,
        "poll_size": 1000,
        "voters": 6756000,
        "electoral": 19,
    },
    {
        "state": "Michigan",
        "dem": 48.3,
        "rep": 46.4,
        "poll_size": 800,
        "voters": 4994000,
        "electoral": 15,
    },
    {
        "state": "Georgia",
        "dem": 48.4,
        "rep": 47.3,
        "poll_size": 800,
        "voters": 4888000,
        "electoral": 16,
    },
    {
        "state": "North Carolina",
        "dem": 48.2,
        "rep": 48.5,
        "poll_size": 800,
        "voters": 4780000,
        "electoral": 16,
    },
    {
        "state": "Wisconsin",
        "dem": 49.5,
        "rep": 46.2,
        "poll_size": 800,
        "voters": 3253000,
        "electoral": 10,
    },
    {
        "state": "Arizona",
        "dem": 47.9,
        "rep": 48.4,
        "poll_size": 800,
        "voters": 3649000,
        "electoral": 11,
    },
    {
        "state": "Nevada",
        "dem": 48.5,
        "rep": 47.7,
        "poll_size": 800,
        "voters": 1351000,
        "electoral": 6,
    },
]

# Scale up the percentages to add to 100
# For each state, give it a posterior distribution of possible dem vote outcomes
for state in states:
    total = state["dem"] + state["rep"]
    state["dem"] = state["dem"] / total * 100
    state["rep"] = state["rep"] / total * 100

    x_observed = int(state["dem"] * state["poll_size"] / 100)
    k_values = np.arange(0, state["voters"] + 1)
    likelihood = binom.pmf(x_observed, state["poll_size"], k_values / state["voters"])
    state["posterior"] = likelihood / np.sum(likelihood)


# Sample a population_p via binomial distribution
def sampleState(state):
    k_values = np.arange(0, state["voters"] + 1)
    sample = np.random.choice(k_values, p=state["posterior"])
    return sample / state["voters"]


def oneOffProbability(state):
    if (state["voters"] % 2) == 0:
        return (
            state["posterior"][int(state["voters"] / 2 - 1)]
            + state["posterior"][int(state["voters"] / 2 + 1)]
        )

    # odd
    return (
        state["posterior"][int(state["voters"] / 2)]
        + state["posterior"][int(state["voters"] / 2 + 1)]
    )


def rangeProbability(state, lower, upper):
    lower = int(lower * state["voters"])
    upper = int(upper * state["voters"])
    return np.sum(state["posterior"][lower:upper])


def sampleElection(states_data):
    # For each state, sample a population_p
    sample_p = {}
    for state in states_data:
        sample_p[state["state"]] = sampleState(state)
    sorted_states = sorted(sample_p.items(), key=lambda x: x[1], reverse=True)
    return sorted_states

In [311]:
tipping_points = {}
state_dict = {state["state"]: state for state in states}
for i in tqdm(range(30000)):
    sample = sampleElection(states)
    votes = 227

    # Go through each state and add it to votes
    # If the state has p > 0.5, next state has p < 0.5, and we would cross 270, then we have a tipping point
    # If the state has p < 0.5, prev state has p > 0.5, and we would cross 270, then we have a tipping point
    tipping_point = ""
    for i in range(len(sample)):
        state, p = sample[i]
        votes += state_dict[state]["electoral"]
        if votes >= 270:
            if p > 0.5 and sample[i + 1][1] < 0.5:
                tipping_point = state
            elif p < 0.5 and sample[i - 1][1] > 0.5:
                tipping_point = state
            break
    if not tipping_point:
        continue
    min_bound = 0
    max_bound = 1

    # Go through the states. If adding tipping point would make the candidate win, then we know max_bound
    # Repeat for reversed states and min_bound
    harris_votes = 227
    for state, p in sample:
        harris_votes += state_dict[state]["electoral"]
        if harris_votes + state_dict[tipping_point]["electoral"] >= 270:
            max_bound = p
            break

    trump_votes = 218
    for state, p in reversed(sample):
        trump_votes += state_dict[state]["electoral"]
        if trump_votes + state_dict[tipping_point]["electoral"] >= 270:
            min_bound = p
            break

    # Find the probability somone wins by 1 vote in this state
    one_off_p = oneOffProbability(state_dict[tipping_point])
    range_p = rangeProbability(state_dict[tipping_point], min_bound, max_bound)
    my_vote_tips_state_given_state_tips_election = one_off_p / range_p

    # Add to tipping_points
    if tipping_point not in tipping_points:
        tipping_points[tipping_point] = []
    tipping_points[tipping_point].append(my_vote_tips_state_given_state_tips_election)

100%|██████████| 30000/30000 [2:12:43<00:00,  3.77it/s]  


In [314]:
# Average the probabilities, divide by num
tipping_points_stats = {}
for state, probs in tipping_points.items():
    tipping_points_stats[state] = {
        "prob_vote_tips_states": np.mean(probs),
        "num": len(probs),
    }

print(
    "Relative probability that your vote tips the state if the state tips the election:"
)
min_prob = min([v["prob_vote_tips_states"] for v in tipping_points_stats.values()])
for state, stats in tipping_points_stats.items():
    print(f"{state}: {stats['prob_vote_tips_states'] / min_prob:.2f}")
print()

# Normalize num by number of samples
print("Probability that state will tip the election:")
num_samples = np.sum([v["num"] for v in tipping_points_stats.values()])
for state, stats in tipping_points_stats.items():
    stats["num"] /= num_samples
    print(f"{state}: {stats['num']:.2f}")
print()

# Multiply to get prob_vote_tips_election
for state, stats in tipping_points_stats.items():
    stats["prob_vote_tips_election"] = stats["prob_vote_tips_states"] * stats["num"]

# Divide by smallest prob_vote_tips_election and print
print("Relative probability that your vote will tip the election:")
min_prob = min([v["prob_vote_tips_election"] for v in tipping_points_stats.values()])
for state, stats in tipping_points_stats.items():
    print(f"{state}: {stats['prob_vote_tips_election'] / min_prob:.2f}")

Relative probability that your vote tips the state if the state tips the election:
Wisconsin: 3.44
North Carolina: 2.03
Pennsylvania: 1.00
Michigan: 2.01
Arizona: 7.21
Georgia: 1.78
Nevada: 23.06

Probability that state will tip the election:
Wisconsin: 0.10
North Carolina: 0.14
Pennsylvania: 0.29
Michigan: 0.15
Arizona: 0.09
Georgia: 0.16
Nevada: 0.07

Relative probability that your vote will tip the election:
Wisconsin: 1.16
North Carolina: 1.00
Pennsylvania: 1.03
Michigan: 1.02
Arizona: 2.24
Georgia: 1.01
Nevada: 5.77
